In [1]:
! pip install -q lmdb fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.8/297.8 kB 27.1 MB/s eta 0:00:00


In [2]:
! git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (225/225), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 499 (delta 208), reused 200 (delta 200), pack-reused 274 (from 1)
Receiving objects: 100% (499/499), 3.05 MiB | 31.59 MiB/s, done.
Resolving deltas: 100% (308/308), done.


In [3]:
%cd deep-text-recognition-benchmark

/content/deep-text-recognition-benchmark


In [4]:
! gdown 1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM

Downloading...
From (original): https://drive.google.com/uc?id=1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM
From (redirected): https://drive.google.com/uc?id=1ubkg7E2vGEOqS4K_quwf9Vl-i8IVpklM&confirm=t&uuid=e0dc0d75-b3ed-4f00-bb42-478793c0347a
To: /content/deep-text-recognition-benchmark/plate_img-train.zip
100% 196M/196M [00:04<00:00, 40.3MB/s]


In [5]:
! gdown 1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I

Downloading...
From (original): https://drive.google.com/uc?id=1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I
From (redirected): https://drive.google.com/uc?id=1AL5Zsg2hDqcwF8ZmR0MJTbjgXIoE5W-I&confirm=t&uuid=de048b51-5249-469e-a97c-e20e82d16733
To: /content/deep-text-recognition-benchmark/plate_img-validation.zip
100% 27.2M/27.2M [00:00<00:00, 49.8MB/s]


In [ ]:
!unzip /content/deep-text-recognition-benchmark/plate_img-train.zip -d /content/Dataset/
!unzip /content/deep-text-recognition-benchmark/plate_img-validation.zip -d /content/Dataset/

```python
def _accumulate(iterable):
    'Return running totals'
    total = 0
    for value in iterable:
        total += value
        yield total
```

In [6]:
!gdown 1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9

Downloading...
From (original): https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9
From (redirected): https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9&confirm=t&uuid=6ceabb3c-69cd-4c07-b468-06b371e023e5
To: /content/deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth
100% 199M/199M [00:04<00:00, 47.2MB/s]


In [7]:
!python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder demo_image/ \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from TPS-ResNet-BiLSTM-Attn.pth
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
demo_image/demo_1.png    	available                	0.9999
demo_image/demo_2.jpg    	shakeshack               	0.9530
demo_image/demo_3.png    	london                   	0.9840
demo_ima

In [21]:
import os
import xml.etree.ElementTree as ET

train_dir = '/content/Dataset/train'
output_file = '/content/deep-text-recognition-benchmark/data/gt_train.txt'
lines = []

for filename in os.listdir(train_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(train_dir, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        image_name = root.find('filename').text
        image_path = os.path.join(train_dir, image_name)

        chars = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            chars.append((xmin, name))

        # Sort characters from left to right
        chars = sorted(chars, key=lambda x: x[0])
        label = ''.join([char[1] for char in chars])

        lines.append(f'{image_path}\t{label}')

# Save to gt.txt
with open(os.path.join(train_dir, output_file), 'w', encoding='utf-8') as f:
    for line in lines:
        f.write(line + '\n')

print(f'Done! {len(lines)} items written to {output_file}')


Done! 19381 items written to /content/deep-text-recognition-benchmark/data/gt_train.txt


In [22]:

import os
import xml.etree.ElementTree as ET

train_dir = '/content/Dataset/validation'
output_file = '/content/deep-text-recognition-benchmark/data/gt_validation.txt'
lines = []

for filename in os.listdir(train_dir):
    if filename.endswith('.xml'):
        xml_path = os.path.join(train_dir, filename)
        tree = ET.parse(xml_path)
        root = tree.getroot()

        image_name = root.find('filename').text
        image_path = os.path.join(train_dir, image_name)

        chars = []
        for obj in root.findall('object'):
            name = obj.find('name').text
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            chars.append((xmin, name))

        # Sort characters from left to right
        chars = sorted(chars, key=lambda x: x[0])
        label = ''.join([char[1] for char in chars])

        lines.append(f'{image_path}\t{label}')

# Save to gt.txt
with open(os.path.join(train_dir, output_file), 'w', encoding='utf-8') as f:
    for line in lines:
        f.write(line + '\n')

print(f'Done! {len(lines)} items written to {output_file}')


Done! 2805 items written to /content/deep-text-recognition-benchmark/data/gt_validation.txt


In [23]:
import os
import shutil

source_dir = '/content/Dataset/train'
destination_dir = '/content/deep-text-recognition-benchmark/data/train/'

os.makedirs(destination_dir, exist_ok=True)

for filename in os.listdir(source_dir):
    if filename.lower().endswith('.jpg'):
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(destination_dir, filename)
        shutil.move(src_path, dst_path)

print("Done train data!")

source_dir = '/content/Dataset/validation'
destination_dir = '/content/deep-text-recognition-benchmark/data/validation'

os.makedirs(destination_dir, exist_ok=True)

for filename in os.listdir(source_dir):
    if filename.lower().endswith('.jpg'):
        src_path = os.path.join(source_dir, filename)
        dst_path = os.path.join(destination_dir, filename)
        shutil.move(src_path, dst_path)

print("Done validation data !")


Done train data!
Done validation data !


In [24]:
!python3 create_lmdb_dataset.py --inputPath data/ --gtFile data/gt_train.txt --outputPath dataset/train

Streaming output truncated to the last 5000 lines.
/content/Dataset/train/for_xml_120743.jpg does not exist
/content/Dataset/train/for_xml_133171.jpg does not exist
/content/Dataset/train/for_xml_134869.jpg does not exist
/content/Dataset/train/for_xml_136475.jpg does not exist
/content/Dataset/train/for_xml_118232.jpg does not exist
/content/Dataset/train/for_xml_122547.jpg does not exist
/content/Dataset/train/for_xml_2163.jpg does not exist
/content/Dataset/train/for_xml_124716.jpg does not exist
/content/Dataset/train/for_xml_119733.jpg does not exist
/content/Dataset/train/for_xml_137853.jpg does not exist
/content/Dataset/train/for_xml_128345.jpg does not exist
/content/Dataset/train/for_xml_120866.jpg does not exist
/content/Dataset/train/for_xml_129256.jpg does not exist
/content/Dataset/train/for_xml_137745.jpg does not exist
/content/Dataset/train/for_xml_123037.jpg does not exist
/content/Dataset/train/for_xml_121844.jpg does not exist
/content/Dataset/train/for_xml_3936.jpg

In [25]:
!python3 create_lmdb_dataset.py --inputPath data/ --gtFile data/gt_validation.txt --outputPath dataset/validation

/content/Dataset/validation/for_xml_117568.jpg does not exist
/content/Dataset/validation/for_xml_135875.jpg does not exist
/content/Dataset/validation/for_xml_125856.jpg does not exist
/content/Dataset/validation/for_xml_6567.jpg does not exist
/content/Dataset/validation/for_xml_3279.jpg does not exist
/content/Dataset/validation/for_xml_131670.jpg does not exist
/content/Dataset/validation/for_xml_121970.jpg does not exist
/content/Dataset/validation/for_xml_133001.jpg does not exist
/content/Dataset/validation/for_xml_121615.jpg does not exist
/content/Dataset/validation/for_xml_4314.jpg does not exist
/content/Dataset/validation/for_xml_135833.jpg does not exist
/content/Dataset/validation/for_xml_130330.jpg does not exist
/content/Dataset/validation/for_xml_133674.jpg does not exist
/content/Dataset/validation/for_xml_135360.jpg does not exist
/content/Dataset/validation/for_xml_135065.jpg does not exist
/content/Dataset/validation/for_xml_128732.jpg does not exist
/content/Datas

In [26]:
!python3 train.py \
--train_data dataset/train --valid_data dataset/validation \
--select_data / --batch_ratio 1 --batch_max_length 8 --valInterval 100 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
dataset_root:    dataset/validation	 dataset: /
sub-directory:	/.	 num samples: 0
Traceback (most recent call last):
  File "/content/deep-text-recognition-benchmark/train.py", line 317, in <module>
    train(opt)
  File "/content/deep-text-recognition-benchmark/train.py", line 36, in train
    valid_loader = torch.utils.data.DataLoader(
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 383, in __init__
    sampler = RandomSampler(dataset, generator=generator)  # type: ignore[arg-type]
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/sampler.py", line 165, in __init__
    raise ValueError(
ValueError: num_samples should be a positive integer value, but got num_samples=0
